# Create a forge and execute an evolution cycle
In this notebook, we will go through the very few steps needed to run a forge cycle for a given budget. 
Before running this notebook, you must set up a few things:

**Your access to LLM's providers:**

To try Ebiose, you must have access to at least one LLM provider. To provide the credentials, copy-paste the [`model_endpoints_template.yml`](./../model_endpoints_template.yml) file, rename it as `model_endpoints.yml` and fill it with your own credentials.

**Add the root directory to your Python path**

Depending on your settings, you may need to add the root of the repository to your `PYTHONPATH` environment variable. You may also use a `.env` file to do so. copy-paste the [`.env.template`](./../.env.template) file, rename it as `.env` and fill it with your own root directory.

**Use LangFuse for tracing**

Ebiose has chosen LangFuse to provide easy and free observability, through its self-hosted capability. Refer to [Langfuse official documentation](https://langfuse.com/self-hosting) to set it up. Once done, fill `LANGFUSE_SECRET_KEY`, `LANGFUSE_PUBLIC_KEY` and `LANGFUSE_HOST` in the `.env` file.

Other observability tools might be used but are not configured yet.

**Load .env file**

To load the `.env` file, execute:

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Creating a basic forge

In ebiose, a **forge** is where custom agents are created to solve specific problems. The forge is the exclusive origin of new agents. Within each forge, architects agents orchestrate the creation and improvement of agents by reusing existing building blocks from the ecosystem.

To create a forge and run a cycle, you must provide the following:
- a description of the forge, which defines the problem that must be solved by generated agents;
- the expected format of the agent's input and output, defined as Pydantic models;
- an implementation of the `compute_fitness` abstract method that will be used by the forge to evaluate the generated agents.

Let's say we wich to generate agents specialized in solving math problems. The forge description could be:

In [2]:
forge_description = "Solving math word problems"

Next, we need to define the expected input and output formats of the generated agents. These formats are to be defined as Pydantic models. 

For instance, in our context of solving math problems, we want the agent input to be a string which will represent the math problem to be solved and the agent output to be composed of two fields:
- `solution` which will be the final solution to the math problem, given as an integer;
- `rationale` which will be the rationale behind the found solution.

The IO Pydantic models will thus be:

In [3]:
from pydantic import BaseModel

class AgentInput(BaseModel):
        math_problem: str

class AgentOutput(BaseModel):
    solution: int
    rationale: str

Lastly, we must provide a way of evaluating the generated agents through the implementation of the `compute_fitness` abstract method of `AgentForge` class. For the sake of demonstration, we will here return a random float between 0 and 1, so that we don't spend tokens at evaluation.

In [4]:
import random
random.seed(7)

from ebiose.core.agent import Agent
from ebiose.core.agent_forge import AgentForge

class BasicForge(AgentForge):
    async def compute_fitness(self, agent: Agent, compute_token_id: str, **kwargs: dict[str, any]) -> float:
        return random.random()

/Users/xabier/dev/ebiose-core/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


We can now instantiate the forge with the provided elements:

In [5]:
forge = BasicForge(
    name="Basic forge",
    description=forge_description,
    agent_input_model=AgentInput,
    agent_output_model=AgentOutput,
    default_generated_agent_engine_type="langgraph_engine",
    default_model_endpoint_id="azure-gpt-4o-mini"
)

# Running a forge cycle

Once the forge is instantiated, we can start generating agents by running a **forge cycle**. 

To do so, you must define:
- a budget in dollars (the forge cycle will end once this budget is exhausted);
- optionally, a path in which created agents and fitness will be saved accross generations. 

> ⚠️ Note that we need to use `asyncio.run` to launch the forge cycle.

> 🚨 Before executing the following cell, check the amount of budget you have allocated!

> 💡 If you are using VSCode, install the [*Markdown Preview Mermaid Support* extension](https://marketplace.visualstudio.com/items?itemName=bierner.markdown-mermaid) to allow the display of the generated agent's graphs.

In [6]:
import asyncio
import nest_asyncio

from ebiose.core.evo_forging_cycle import EvoForgingCylceConfig
nest_asyncio.apply()

from pathlib import Path
from datetime import UTC, datetime

# the path where results will be saved
current_time = datetime.now(UTC).strftime("%Y-%m-%d_%H-%M-%S")
SAVE_PATH = Path(f"./../data/") / current_time
if not SAVE_PATH.exists():
    SAVE_PATH.mkdir(parents=True)


cycle_config = EvoForgingCylceConfig(
    budget=0.01, # the budget for the forge cycle in dollas
    n_agents_in_population=2, # number of agents in the population, at each generation
    n_selected_agents_from_ecosystem=0, # number of agents selected from the ecosystem at initialization
    n_best_agents_to_return=2, # number of best agents to return at the end of the cycle
    replacement_ratio=0.5, # ratio of agents replaced at each generation
    save_path=SAVE_PATH, # the path where results will be saved (optional)
)

final_agents, final_fitness = asyncio.run(
    forge.run_new_cycle(config=cycle_config)
)


Starting a new cycle for forge Basic forge
****** Initializing agents population ******
Creating 2 new agents with architect agents...
  0%|          | 0/2 [00:00<?, ?it/s]
Initializing structured output agent for model AgentOutput (1)
100%|██████████| 2/2 [00:22<00:00, 11.34s/it]
Agent initialization cost: 0.006066885
Population initialized with 2 agents
Initialization of 2 agents took 0:00:22.713719
Budget left after initialization: 0.003933115 $
****** Running generation 0 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 2789.69it/s]
Agent agent-36a33679-be06-4249-8504-77537d9e14ef fitness: 0.45318437637077535, cost: 0.0
Agent agent-81110ef2-e25a-4778-9074-31486c138f40 fitness: 0.29976699686368236, cost: 0.0
Evaluation took 0:00:00.003377 for a total cost of 0.0 $


# Agent ID: agent-36a33679-be06-4249-8504-77537d9e14ef
## Fitness: 0.45318437637077535
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node(Math Problem Understanding LLM)
	Llm_Node(Math Problem Understanding LLM) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of an AI model designed to collaboratively solve math word problems. Each node in this network has a specific role in the problem-solving process. Your task is to process information, draw insights, and communicate effectively with other nodes. The Math Problem Understanding LLM node will analyze the input problem, extracting key information and formulating an approach. You must ensure that your responses are clear, logical, and conducive to further problem-solving steps. Each interaction with the next node should provide all necessary information derived from your analysis.
##### Math Problem Understanding LLM
Analyze the following math word problem and identify the key elements necessary for solving it. Break down the problem into its components, such as the quantities involved, the operations required, and any relationships between the elements. Use chain-of-thought reasoning to articulate your understanding clearly. After your analysis, summarize the approach you recommend for solving the problem, including any assumptions you make. Finally, reflect on whether the problem has any ambiguities that need clarification, and prepare a concise summary of your findings to pass on to the next node. 

Here is the math word problem: 'A farmer has 20 apples. He gives 5 apples to his friend and then buys 10 more apples. How many apples does he have now?'

# Agent ID: agent-81110ef2-e25a-4778-9074-31486c138f40
## Fitness: 0.29976699686368236
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(Information Extractor)
	Llm_Node_1(Information Extractor) --> Llm_Node_2(Solution Generator)
	Llm_Node_2(Solution Generator) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems. Each node in this system has a specific role: 
1. The Information Extractor identifies and extracts key information and variables from the word problem.
2. The Solution Generator creates a structured mathematical representation and outlines a potential solution approach.

Work together to ensure a seamless flow of information. Pay attention to the outputs of the previous node and leverage your unique capabilities to enhance the problem-solving process. Provide clear and comprehensive outputs that can be easily understood by the next node in the sequence.
##### Information Extractor
Your task is to read the provided math word problem and extract essential information, including numbers, operations, and relationships between variables.

1. Identify key elements of the problem: What are the main subjects involved? What quantities are given, and what is being asked?
2. Clearly list the relevant information you find, including any mathematical terms or phrases that indicate operations (e.g., sum, difference, product, etc.).
3. Ensure that you present the extracted information in a structured format that the next node can easily interpret.

For example, if the problem is: "Alice has 5 apples, and Bob gives her 3 more. How many apples does she have now?" your output should indicate:
- Subjects: Alice, Bob
- Given quantities: 5 apples (Alice), 3 apples (Bob)
- Operation: Addition (5 + 3)
- Question: How many apples does Alice have now?

Please proceed with the extraction process now.
##### Solution Generator
Your role is to take the information provided by the Information Extractor and formulate a structured mathematical representation of the problem, along with a proposed solution strategy.

1. Begin by reviewing the extracted information from the previous node. Ensure that you understand the key variables and the relationships identified.
2. Create a mathematical equation or model based on the information provided. Clearly articulate the steps needed to arrive at the solution.
3. Explain your reasoning and decision-making process. If necessary, provide alternative methods or approaches to solve the problem.
4. Clearly present the final answer to the problem, ensuring it is well-formulated and easy to understand.

For example, if the extracted information indicated: 
- Subjects: Alice, Bob
- Given quantities: 5 apples, 3 apples
- Operation: Addition
You might respond with: "To find the total number of apples Alice has, we can express this as an equation: Total apples = 5 + 3. Therefore, Alice has 8 apples."

Please generate the structured mathematical representation and the solution now.



Saving current state to ../data/2025-03-05_21-36-46/generation=0
Starting crossover and mutation...

Initializing structured output agent for model AgentOutput (2)
Number of offsprings: 1/1
Crossover and mutation completed in 0:00:14.930166 for a total cost of 0.00157839 $
Generation 0 completed in 0:00:14.978140 with a total cost of 0.00157839 $
Budget left after first generation: 0.0023547250000000002 $
****** Running generation 1 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 2864.96it/s]
Agent agent-81110ef2-e25a-4778-9074-31486c138f40 fitness: 0.03920725704743766, cost: 0.0
Agent agent-0fbe36fc-3c48-4c6b-8ec6-b98d72328505 fitness: 0.6682158565343952, cost: 0.0
Evaluation took 0:00:00.003313 for a total cost of 0.0 $


# Agent ID: agent-0fbe36fc-3c48-4c6b-8ec6-b98d72328505
## Fitness: 0.6682158565343952
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(Math Problem Understanding LLM)
	Llm_Node_1(Math Problem Understanding LLM) --> Llm_Node_2(Math Problem Solution LLM)
	Llm_Node_2(Math Problem Solution LLM) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of an AI model designed to collaboratively solve math word problems. Each node in this network has a specific role in the problem-solving process. Your task is to process information, draw insights, and communicate effectively with other nodes. The Math Problem Understanding LLM node will analyze the input problem, extracting key information and formulating an approach. The Math Problem Solution LLM will compute the answer based on the first node's analysis. You must ensure that your responses are clear, logical, and conducive to further problem-solving steps. Each interaction with the next node should provide all necessary information derived from your analysis.
##### Math Problem Understanding LLM
Analyze the following math word problem and identify the key elements necessary for solving it. Break down the problem into its components, such as the quantities involved, the operations required, and any relationships between the elements. Use chain-of-thought reasoning to articulate your understanding clearly. After your analysis, summarize the approach you recommend for solving the problem, including any assumptions you make. Finally, reflect on whether the problem has any ambiguities that need clarification, and prepare a concise summary of your findings to pass on to the next node. 

Here is the math word problem: 'A farmer has 20 apples. He gives 5 apples to his friend and then buys 10 more apples. How many apples does he have now?'
##### Math Problem Solution LLM
Using the analysis provided, compute the final answer to the math word problem. Ensure that all steps are clearly shown, and check for any potential mistakes or misunderstandings in the previous analysis. Provide a clear explanation of how you arrived at the answer.

# Agent ID: agent-81110ef2-e25a-4778-9074-31486c138f40
## Fitness: 0.03920725704743766
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(Information Extractor)
	Llm_Node_1(Information Extractor) --> Llm_Node_2(Solution Generator)
	Llm_Node_2(Solution Generator) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems. Each node in this system has a specific role: 
1. The Information Extractor identifies and extracts key information and variables from the word problem.
2. The Solution Generator creates a structured mathematical representation and outlines a potential solution approach.

Work together to ensure a seamless flow of information. Pay attention to the outputs of the previous node and leverage your unique capabilities to enhance the problem-solving process. Provide clear and comprehensive outputs that can be easily understood by the next node in the sequence.
##### Information Extractor
Your task is to read the provided math word problem and extract essential information, including numbers, operations, and relationships between variables.

1. Identify key elements of the problem: What are the main subjects involved? What quantities are given, and what is being asked?
2. Clearly list the relevant information you find, including any mathematical terms or phrases that indicate operations (e.g., sum, difference, product, etc.).
3. Ensure that you present the extracted information in a structured format that the next node can easily interpret.

For example, if the problem is: "Alice has 5 apples, and Bob gives her 3 more. How many apples does she have now?" your output should indicate:
- Subjects: Alice, Bob
- Given quantities: 5 apples (Alice), 3 apples (Bob)
- Operation: Addition (5 + 3)
- Question: How many apples does Alice have now?

Please proceed with the extraction process now.
##### Solution Generator
Your role is to take the information provided by the Information Extractor and formulate a structured mathematical representation of the problem, along with a proposed solution strategy.

1. Begin by reviewing the extracted information from the previous node. Ensure that you understand the key variables and the relationships identified.
2. Create a mathematical equation or model based on the information provided. Clearly articulate the steps needed to arrive at the solution.
3. Explain your reasoning and decision-making process. If necessary, provide alternative methods or approaches to solve the problem.
4. Clearly present the final answer to the problem, ensuring it is well-formulated and easy to understand.

For example, if the extracted information indicated: 
- Subjects: Alice, Bob
- Given quantities: 5 apples, 3 apples
- Operation: Addition
You might respond with: "To find the total number of apples Alice has, we can express this as an equation: Total apples = 5 + 3. Therefore, Alice has 8 apples."

Please generate the structured mathematical representation and the solution now.



Saving current state to ../data/2025-03-05_21-36-46/generation=1
Starting crossover and mutation...
Number of offsprings: 1/1
Crossover and mutation completed in 0:00:15.501519 for a total cost of 0.0019148250000000002 $
Generation 1 completed in 0:00:15.512495 with a total cost of 0.0019148250000000002 $
Budget left after new generation: 0.0004399 $
100%|██████████| 2/2 [00:00<00:00, 3269.14it/s]
Agent agent-0fbe36fc-3c48-4c6b-8ec6-b98d72328505 fitness: 0.9930959394666341, cost: 0.0
Agent agent-3ad00409-8ad3-4376-b188-3969aa3b05c8 fitness: 0.8219247866097149, cost: 0.0
Saving current state to ../data/2025-03-05_21-36-46/generation=2
Cycle completed in 0:00:53.218038 with a total cost of 0.0095601 $
Budget left at final: 0.0004399 $
Returning 2 best agents


We can now display the best agents that have been returned as follows. Note that:
- all agents can be found in the `SAVE_PATH` directory if you defined one;
- here, the compute fitness only returns a random float, so the following displayed agents have not been truly evaluated. 

Go check [examples/math_forge/math_forge.py](./../examples/math_forge/math_forge.py) to see a fully implemeted forge with a non-random fitness evaluation function.

In [7]:
forge.display_results(final_agents, final_fitness)

# Agent ID: agent-0fbe36fc-3c48-4c6b-8ec6-b98d72328505
## Fitness: 0.9930959394666341
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(Math Problem Understanding LLM)
	Llm_Node_1(Math Problem Understanding LLM) --> Llm_Node_2(Math Problem Solution LLM)
	Llm_Node_2(Math Problem Solution LLM) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of an AI model designed to collaboratively solve math word problems. Each node in this network has a specific role in the problem-solving process. Your task is to process information, draw insights, and communicate effectively with other nodes. The Math Problem Understanding LLM node will analyze the input problem, extracting key information and formulating an approach. The Math Problem Solution LLM will compute the answer based on the first node's analysis. You must ensure that your responses are clear, logical, and conducive to further problem-solving steps. Each interaction with the next node should provide all necessary information derived from your analysis.
##### Math Problem Understanding LLM
Analyze the following math word problem and identify the key elements necessary for solving it. Break down the problem into its components, such as the quantities involved, the operations required, and any relationships between the elements. Use chain-of-thought reasoning to articulate your understanding clearly. After your analysis, summarize the approach you recommend for solving the problem, including any assumptions you make. Finally, reflect on whether the problem has any ambiguities that need clarification, and prepare a concise summary of your findings to pass on to the next node. 

Here is the math word problem: 'A farmer has 20 apples. He gives 5 apples to his friend and then buys 10 more apples. How many apples does he have now?'
##### Math Problem Solution LLM
Using the analysis provided, compute the final answer to the math word problem. Ensure that all steps are clearly shown, and check for any potential mistakes or misunderstandings in the previous analysis. Provide a clear explanation of how you arrived at the answer.

# Agent ID: agent-3ad00409-8ad3-4376-b188-3969aa3b05c8
## Fitness: 0.8219247866097149
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(Math Problem Understanding LLM)
	Llm_Node_1(Math Problem Understanding LLM) --> Llm_Node_2(Math Problem Validation LLM)
	Llm_Node_2(Math Problem Validation LLM) --> Llm_Node_3(Math Problem Solution LLM)
	Llm_Node_3(Math Problem Solution LLM) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of an AI model designed to collaboratively solve math word problems. Each node in this network has a specific role in the problem-solving process. Your task is to process information, draw insights, and communicate effectively with other nodes. The Math Problem Understanding LLM node will analyze the input problem, extracting key information and formulating an approach. The Math Problem Validation LLM will review the analysis for any potential errors. The Math Problem Solution LLM will compute the answer based on the validated analysis. You must ensure that your responses are clear, logical, and conducive to further problem-solving steps. Each interaction with the next node should provide all necessary information derived from your analysis.
##### Math Problem Understanding LLM
Analyze the following math word problem and identify the key elements necessary for solving it. Break down the problem into its components, such as the quantities involved, the operations required, and any relationships between the elements. Use chain-of-thought reasoning to articulate your understanding clearly. After your analysis, summarize the approach you recommend for solving the problem, including any assumptions you make. Finally, reflect on whether the problem has any ambiguities that need clarification, and prepare a concise summary of your findings to pass on to the next node. \n\nHere is the math word problem: 'A farmer has 20 apples. He gives 5 apples to his friend and then buys 10 more apples. How many apples does he have now?'
##### Math Problem Validation LLM
Based on the analysis from the previous step, validate the approach and check for any potential misunderstandings or oversights. Highlight any ambiguities or critical points that need clarification before proceeding to compute the final answer. Provide feedback for improvement if necessary.
##### Math Problem Solution LLM
Using the validated analysis provided, compute the final answer to the math word problem. Ensure that all steps are clearly shown, and check for any potential mistakes or misunderstandings in the previous analysis. Provide a clear explanation of how you arrived at the answer.

